In [1]:
from os import environ
import httpx
import importlib
import pathlib
import polars as pl
import src.fetch
import src.ingest
from dotenv import load_dotenv

load_dotenv()

## Polars
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(100)

polars.config.Config

In [2]:
importlib.reload(src.fetch)
fetcher = src.fetch.Fetch(
    client=httpx.AsyncClient(),
    tiingo_token=environ["TIINGO_TOKEN"],
    add_ticker_column=True,
    min_start_date="1995-01-01",
    save_dir="data/sample2",
    response_format="csv",
    failed_tickers_file="failed_tickers.csv",
)

if "duckdb_con" not in locals():
    duckdb_con = src.fetch.get_supported_tickers()

## TODO: problematic tickers:

- Go through concrete cases and see which exchange they come from, and whether that could be a source to the issue. Consider skipping one of the exchanges if one seems problematic.
- Consider making a function deletes redundant csvs before ingestion. As part of the Ingest class.
  - Recurring pattern: may it be that the later file is always better (but not entire sure about this. Check other file name as well)
  - Pattern: if duplicate tickers, only select the row with the latest end date.
- TODO: move transformations to a Python module and add tests for the transformations

### Concrete cases:

- AWH: seems to be downloaded twice, but with the same data. Omit one of the fetches or skip the one of the exchanges, as that might be the issue.
- BETR: file BETR_2015-08-05_2023-08-29 seems be redundant, as it has only 4 days of data, which overlaps with a more recent file.
- BFLY-WS: BFLY-WS_2020-07-13_2022-12-23 is redundant. The other file is also incomplete, with the first row of data occuring about one year before the second row. After that it looks normal
- CNR: CNR_1995-01-01_2022-07-22 is redundant. Other file also looks suspicious.
- COGT: COGT_2007-12-27_2018-04-13 is redundant. Other file looks good.
- CSR: CSR_2007-10-29_2011-09-15 is redundant. Other file looks good.
- CZR: CZR_2012-02-08_2020-07-20 is redundant. Other file looks good.
- DD: DD_2017-08-03_2019-05-31 is redunant. Other file looks good.

In [3]:
us_tickers_query = pathlib.Path("sql/selected_us_tickers.sql").read_text()
duckdb_con.sql(us_tickers_query)

In [4]:
duckdb_con.sql("select count(*) from selected_us_tickers")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        16945 │
└──────────────┘

In [5]:
df_us_clean = duckdb_con.table("selected_us_tickers").pl()

(16945, 6)

In [6]:
seed = 42
df_sampled = df_us_clean.sample(500, seed=seed)
df_sampled.shape

(500, 6)

In [7]:
await fetcher.fetch_supported_tickers(
    df=df_sampled,
    columns=["date", "adjClose", "adjVolume"],
)

In [13]:
df_zzz = df_us_clean.filter(
    ((pl.col("ticker") == "ZZZ") & (pl.col("exchange") == "NASDAQ"))
    | (pl.col("ticker") == "AAPL")
    | (pl.col("ticker") == "CCAI")
)

await fetcher.fetch_supported_tickers(
    df=df_zzz,
    columns=["date", "adjClose", "adjVolume"],
)

In [10]:
seed=42
df_large = df_us_clean.sample(7022, seed=seed)
df_large.shape

(7022, 6)

In [11]:
await fetcher.fetch_all(
    df=df_large,
    columns=["date", "adjClose", "adjVolume"],
    async_batch_size=400
)

Fetching 0 to 400 of 7022


ERROR:src.fetch:Invalid response for LCI.
ERROR:src.fetch:An error occurred while requesting https://api.tiingo.com/tiingo/daily/DGRS/prices?startDate=2013-07-25&endDate=2024-05-10&format=csv&columns=date,adjClose,adjVolume&token=72b7d80c10cbdbdc839fe9556a7a004af06cf83c: [Errno 8] nodename nor servname provided, or not known
ERROR:src.fetch:response.text for DGRS is None.


Fetching 400 to 800 of 7022


ERROR:src.fetch:Invalid response for AVYA.


Fetching 800 to 1200 of 7022


...

ERROR:src.fetch:No data found for MEME.
ERROR:src.fetch:An error occurred while requesting https://api.tiingo.com/tiingo/daily/PTRB/prices?startDate=2021-12-08&endDate=2024-05-10&format=csv&columns=date,adjClose,adjVolume&token=72b7d80c10cbdbdc839fe9556a7a004af06cf83c: [Errno 8] nodename nor servname provided, or not known
ERROR:src.fetch:response.text for PTRB is None.
ERROR:src.fetch:Invalid response for FOGO.
ERROR:src.fetch:Invalid response for TFG.


Fetching 6400 to 6800 of 7022


ERROR:src.fetch:Invalid response for CIC-U.
ERROR:src.fetch:No data found for RESI.


Fetching 6800 to 7200 of 7022


ERROR:src.fetch:No data found for AMPIW.
ERROR:src.fetch:Invalid response for ABDC.
ERROR:src.fetch:No data found for SAAS.
ERROR:src.fetch:No data found for KEGX.


# Ingest to DuckDB / Motherduck



In [12]:
importlib.reload(src.ingest)
ingestor = src.ingest.Ingest(
    db_path="duckdb/test.db",
    sql_dir="sql",
)
ingestor.create_table(file_name="create_table.sql")

In [13]:
ingestor.ingest_data(dst_table="daily_adjusted", data="data/sample/*.csv")
ingestor.cur.sql("SELECT * FROM daily_adjusted LIMIT 10")

┌────────────┬───────────────┬───────────┬─────────┐
│    date    │   adjClose    │ adjVolume │ ticker  │
│    date    │ decimal(18,3) │  uint64   │ varchar │
├────────────┼───────────────┼───────────┼─────────┤
│ 2015-11-11 │        24.500 │   1584639 │ AAAP    │
│ 2015-11-12 │        25.000 │     83061 │ AAAP    │
│ 2015-11-13 │        25.260 │     67396 │ AAAP    │
│ 2015-11-16 │        25.020 │     16959 │ AAAP    │
│ 2015-11-17 │        24.620 │     25972 │ AAAP    │
│ 2015-11-18 │        25.000 │    111122 │ AAAP    │
│ 2015-11-19 │        25.900 │    113155 │ AAAP    │
│ 2015-11-20 │        25.200 │     60324 │ AAAP    │
│ 2015-11-23 │        25.150 │     59704 │ AAAP    │
│ 2015-11-24 │        25.620 │     56032 │ AAAP    │
├────────────┴───────────────┴───────────┴─────────┤
│ 10 rows                                4 columns │
└──────────────────────────────────────────────────┘

In [14]:
ingestor.cur.sql("SELECT COUNT(*) FROM daily_adjusted")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     15982999 │
└──────────────┘

In [18]:
ingestor.cur.sql("SELECT * FROM daily_adjusted WHERE ticker='ABBV' and date>='2021-01-04' and date<='2021-01-08'")

┌────────────┬───────────────┬───────────┬─────────┐
│    date    │   adjClose    │ adjVolume │ ticker  │
│    date    │ decimal(18,3) │  uint64   │ varchar │
├────────────┼───────────────┼───────────┼─────────┤
│ 2021-01-04 │        91.289 │   9523377 │ ABBV    │
│ 2021-01-05 │        92.233 │   6823826 │ ABBV    │
│ 2021-01-06 │        91.436 │  11017467 │ ABBV    │
│ 2021-01-07 │        92.415 │   8196046 │ ABBV    │
│ 2021-01-08 │        92.900 │   5345883 │ ABBV    │
└────────────┴───────────────┴───────────┴─────────┘

In [26]:
ingestor.close()

In [32]:
response = httpx.get(f"https://api.tiingo.com/tiingo/daily/prices?format=csv&date=2024-05-09&token={environ['TIINGO_TOKEN']}")
response.status_code

200

In [33]:
import pathlib
path = pathlib.Path("data/daily/snapshot_date.csv")
path.write_text(response.text)

3934387